# Homework

Task
* Find the hyperparameters that yield get the best predictions on the test dataset

Team
* Yufei Chu
* Kavish Vijay Daftari

Imoprting the libraries

In [1]:
import numpy as np
from urllib import request
import gzip
import pickle

Code to define two functions, download_mnist() and save_mnist(), that can be used to download and save the MNIST dataset.

In [2]:
filename = [
    ["training_images","train-images-idx3-ubyte.gz"],
    ["test_images","t10k-images-idx3-ubyte.gz"],
    ["training_labels","train-labels-idx1-ubyte.gz"],
    ["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

In [3]:
download_mnist()
save_mnist()

Download complete.
Save complete.


In [4]:
def load():
    with open(r"mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]


Loading MINST dataset and storing the training and test images and labels into separate numpy arrays for further processing.

In [5]:
X_train, y_train, X_test, y_test = load()

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [7]:
X_train.size

47040000

Check if there are any *non-zero* elements in our matrices, use `np.any()`, which returns True if it finds at least one non-zero element:

In [8]:
np.any(X_train), np.any(X_test)

(True, True)

The sparsity of our dataset:

In [9]:
np.count_nonzero(X_train, axis=None) / X_train.size, np.count_nonzero(X_test, axis=None) / X_test.size

(0.19120229591836735, 0.1927575255102041)

In [10]:
np.any(y_train), np.any(y_test)

(True, True)

In [11]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Prepare data so that it is *normalized* and *batched*:

In [12]:
def norm(X, x_min, x_max):
    nom = (X - X.min(axis=0)) * (x_max-x_min)
    denom = X.max(axis=0) - X.min(axis=0)
    denom[denom==0] = 1
    return x_min + nom/denom 

In [13]:
def prep_data(X, y):

    X_ = []
    y_ = []

    itr = int(len(y) / batch_size) + 1
    for j in range(1, itr):
        rng = j * batch_size
        X_.append(X[rng - batch_size : rng, :])
        y_.append(y[rng - batch_size : rng])

    X, y = np.array(X_), np.array(y_)
    X = norm(X, 0, 1)

    return X, y

## Tuning the hyperpatameter learning rates to 0.01

Our hyperparameters: `eta` and `alpha` are the multiplicative and additive **learning rates**, used to put the brakes on *giant* weight updates (baby steps!). `n_iter` is the number of training epochs, and `batch_size` is the size of our minibatch:

In [18]:
eta=0.01
alpha=0.01
n_iter=100
batch_size=50

In [19]:
eta, alpha, n_iter, batch_size

(0.01, 0.01, 100, 50)

Let's prepare our data:

In [20]:
X_train, y_train = prep_data(X_train, y_train)

In [21]:
X_train.shape, y_train.shape

((1200, 50, 784), (1200, 50))

In [22]:
1200 * 50

60000

In [23]:
np.count_nonzero(X_train, axis=None) / X_train.size

0.19120229591836735

One-hot encode our labels so that instead of many categories denoted by integers one through 10, we only have two categories: `0` and `1`, albeit in more dimensions.

In [24]:
def one_hot_enc(y, num_labels=10):
    one_hot = np.zeros((num_labels, y.shape[0]), dtype=np.float32)

    for i, val in enumerate(y):
        one_hot[val,i] = 1.0

    return one_hot

In [25]:
def one_hot_enc_v2(y, num_labels=10):
    one_hot = np.zeros((y.shape[0], num_labels), dtype=np.float32)

    for i, val in enumerate(y):
        one_hot[i,val] = 1.0

    return one_hot

Initialize our weights:

In [26]:
def init_weights(n_input, n_hidden_1, n_hidden_2, n_output, batch_size):
    #w1 = np.random.randn(n_hidden_1, n_input + 1)
    #w2 = np.random.randn(n_hidden_2, n_hidden_1 + 1)
    #w3 = np.random.randn(n_output, n_hidden_2 + 1)
    
    w1 = np.random.randn(n_input, n_hidden_1)
    w2 = np.random.randn(n_hidden_1, n_hidden_2)
    w3 = np.random.randn(n_hidden_2, n_output)
    
    return w1, w2, w3

Binary cross-entropy loss:

In [27]:
def compute_loss(prediction, label):
    term_1 = -1*label * np.log(prediction)
    term_2 = (1 - label) * (np.log(1 - prediction))

    loss = np.sum(term_1 - term_2)
    return loss

Initialize the number of neurons per layer:

In [28]:
n_hidden_1, n_hidden_2, n_output = 100, 100, 10
n_input = len(X_train[0,0,:]) #returns the flattened image size (28*28 = 784)
n_input

784

Initialize the weight1, weight2, weight3

In [29]:
w1, w2, w3 = init_weights(n_input, n_hidden_1, n_hidden_2, n_output, batch_size)

Initialize Markovian weight history, losses and accuracy per epoch:

In [30]:
delta_w1_prev = np.zeros(w1.shape)
delta_w2_prev = np.zeros(w2.shape)
delta_w3_prev = np.zeros(w3.shape)

train_losses = []
train_acc = []

Here is the full code for the **feedforward pass** (we return input and all outputs):

In [31]:
def compute_forward_pass(inputs):   
    a1 = inputs

    # outputs of first hidden layer
    z2 = np.matmul(a1, w1)
    a2 = 1/(1 + np.exp(-z2))

    # outputs of second hidden layer
    z3 = np.matmul(a2, w2)
    a3 = 1/(1 + np.exp(-z3))

    # outputs of final layer
    z4 = np.matmul(a3, w3)
    a4 = 1/(1 + np.exp(-z4))
    
    return a1, z2, a2, z3, a3, z4, a4

In **backpropagation**, we evaluate the gradients of the loss with respect to the weights of each layer. 

The **gradient descent algorithm** consists in subtracting the respective gradient from each layer's weights (subject to an additive and multiplicative "*brake*" in order to take baby steps in that direction) so that we may reach the point where we minimize the difference between labels and predictions.

<br />
<center>
<img src =ipynb.images/duck-baby-steps.png width = 400 />
</center>

## 2 Hidden layers equations
Our equations for two hidden layers are ($\hat{y}$ is the output of the network while $y$ is the label, $z$ is the output of hidden layer 2, $r$ is the output of hidden layer 1, and $x$ is the input):

$$\frac{\partial \;\text{Loss}(y, \hat{y})}{\partial W^3} = 
2(y - \hat{y}) * \sigma'(W^3 z) \otimes z$$

$$\frac{\partial \;\text{Loss}(y, \hat{y})}{\partial W^2} = 
2(y - \hat{y}) * \sigma'(W^3 z)  \otimes W^3 *
\sigma'(W^2 r) \otimes r$$

$$\frac{\partial \;\text{Loss}(y, \hat{y})}{\partial W^1} =
2(y - \hat{y}) * \sigma'(W^3 z) \otimes W^3 
* \sigma'(W^2 r) \otimes W^2 * \sigma'(W^1 x) \otimes x$$

`a` for activation: `a1` is the input data, `a2` is the output of hidden layer 1, `a3` is the output of hidden layer 2, `a4` is the output of the final layer.

`z` for linear data: `z1` is the input data, `z2` is the linear output of hidden layer 1 *before its activation function*, `z3` is the linear output of hidden layer 2 *before its activation function*, `z4` is the linear output of the final layer *before its activation function*.

In analogy with our `ml_dense_pruned` notebook, $\hat{y}$ is the output of the network while $y$ is the label, $z$ is the output of hidden layer 2, $r$ is the output of hidden layer 1, and $x$ is the input. In code, $z = $ `self.layer2` is the output of hidden layer 2, and $r = $ `self.layer1` is the output of hidden layer 1. 

$\sigma'(W^3 z) = $ `sigmoid_derivative_3`.

$\sigma'(W^2 r) = $ `sigmoid_derivative_2`.

$\sigma'(W^1 z) = $ `sigmoid_derivative_1`.

Here's our activation function and its derivative:

In [32]:
def sigmoid(x):
    try:
        x = np.vectorize(round)(x)
        return 1.0/(1+ np.exp(-x))
    except:
        print(x)
        print(np.exp(-x))
        return 1.0/(1+ np.exp(-x))

In [33]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

Backpropagation pass:

In [34]:
def compute_backward_pass_fullmath(outputs, label, verbose=False):
    a1, z2, a2, z3, a3, z4, a4 = outputs

    #
    # LAST LAYER
    #
    
    # from ml-dense-pruned:
    # sigmoid_derivative_3 = sigmoid_derivative(np.dot(self.layer2, self.weights3)) # s'(W3 z) derivative of final layer output
    # d_weights3 = np.dot(self.layer2.T, 
    #                        (2*(self.y - self.output) * sigmoid_derivative_3))
    sigmoid_derivative_3 = sigmoid_derivative(np.dot(a3, w3)) # s'(W3 z) derivative of final layer output [50, 10]
    if verbose: print("sigmoid_derivative_3:", sigmoid_derivative_3.shape, "should be [50, 10]")
    d_weights3 = np.dot(a3.T, 
                           (2 * (label - a4) * sigmoid_derivative_3)) # [50, 100].T * [50, 10]
    if verbose: print("d_weights3:", d_weights3.shape, "should be [100, 10]")
    
    
    # 
    # 2ND HIDDEN LAYER
    #

    # from ml-dense-pruned:
    # sigmoid_derivative_2 = sigmoid_derivative(np.dot(self.layer1, self.weights2)) # s'(r W2) derivative of hidden layer 2 out
    # d_weights2 = np.dot(self.layer1.T,  
    #                        np.dot(2*(self.y - self.output) * sigmoid_derivative_3, self.weights3.T) * 
    #                        sigmoid_derivative_2)
    sigmoid_derivative_2 = sigmoid_derivative(np.dot(a2, w2)) # s'(r W2) derivative of hidden layer 2 out
    if verbose: print("sigmoid_derivative_2:", sigmoid_derivative_2.shape, "should be [50, 100]")
    d_weights2 = np.dot(a2.T,  
                        np.dot((2 * (label - a4) * sigmoid_derivative_3), w3.T) * 
                        sigmoid_derivative_2) # [50, 100].T * ([50, 10] * [100, 10].T)[50, 100] = [50, 100].T * [50, 100] 
                                                                                                # = [100, 100]
    if verbose: print("d_weights2:", d_weights2.shape, "should be [100, 100]")
    
    
    # 
    # 1ST HIDDEN LAYER
    #
    
    # from ml-dense-pruned:
    # sigmoid_derivative_1 = sigmoid_derivative(np.dot(self.input, self.weights1))  # s'(x W1) derivative of hidden layer 1 out
    # d_weights1 = np.dot(self.input.T,  
    #                        np.dot(
    #                            np.dot(2*(self.y - self.output) * sigmoid_derivative_3, self.weights3.T) *
    #                                sigmoid_derivative_2, self.weights2.T) *   
    #                        sigmoid_derivative_1)
    sigmoid_derivative_1 = sigmoid_derivative(np.dot(a1, w1))  # s'(x W1) derivative of hidden layer 1 out [50, 100]
    if verbose: print("sigmoid_derivative_1:", sigmoid_derivative_1.shape, "should be [50, 100]")
    d_weights1 = np.dot(a1.T,  
                           np.dot(
                               np.dot((2 * (label - a4) * sigmoid_derivative_3), w3.T) *
                                   sigmoid_derivative_2, w2.T) *   
                           sigmoid_derivative_1) # [50, 784].T * (([50, 100] * [100, 100].T) . [50, 100]) * [100, 100].T)
                                                # = [50, 784].T * [50, 100] * [100, 100].T
                                                # = [50, 784].T * [50, 100].T
                                                # = [784, 100]
    if verbose: print("d_weights1:", d_weights1.shape, "should be [784, 100]")

    return d_weights1, d_weights2, d_weights3

Predict from the output of 10 neurons, representing the digit predicted in a one-hot encoded fashion:

In [35]:
def predict(a4):
    prediction = np.argmax(a4, axis=1)
    return prediction

The training dataset:

In [36]:
X_train.shape, y_train.shape

((1200, 50, 784), (1200, 50))

In [37]:
X_train[:, 0, ]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
np.any(X_train)

True

In [39]:
y_train

array([[5, 0, 4, ..., 5, 9, 3],
       [3, 0, 7, ..., 8, 3, 1],
       [5, 7, 1, ..., 6, 7, 3],
       ...,
       [5, 7, 4, ..., 2, 0, 9],
       [4, 0, 1, ..., 5, 9, 8],
       [8, 4, 0, ..., 5, 6, 8]], dtype=uint8)

This is how to iterate over the training dataset:

Let's debug *one batch* of one epoch:

## Set the hyperparameter learning rate = 0.01, epoch = 100:

In [40]:
eta=0.01
alpha=0.01
batch_size=50
n_iter = 100

In [ ]:
from tqdm import tqdm
#epoch loop
for i in tqdm(range(n_iter)):
    # batch loop
    for (input, label) in zip(X_train, y_train):
        one_hot_label = one_hot_enc_v2(label, num_labels=10)

        a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(input)
        loss = compute_loss(a4, one_hot_label)
        grad1, grad2, grad3 = compute_backward_pass_fullmath([a1, z2, a2, z3, a3, z4, a4], one_hot_label)

        # multiplicative learning factor
        delta_w1, delta_w2, delta_w3 = eta * grad1, eta * grad2, eta * grad3

        # additive learning factor
        w1 = w1 + delta_w1 + delta_w1_prev * alpha
        w2 = w2 + delta_w2 + delta_w2_prev * alpha
        w3 = w3 + delta_w3 + delta_w3_prev * alpha

        delta_w1_prev, delta_w2_prev, delta_w3_prev = delta_w1, delta_w2, delta_w3

        train_losses.append(loss)
        predictions = predict(a4)

        wrong = np.where(predictions != label, np.matrix([1.]), np.matrix([0.]))
        accuracy = 1 - (np.sum(wrong) / batch_size)
        train_acc.append(accuracy)

    # epoch loss and accuracy (mean of al batches)
    print('epoch ', i, 'loss %.2f' % np.mean(np.matrix(train_losses)).item(), 
          'training accuracy %.2f' % np.mean(np.matrix(train_acc)).item())

  1%|          | 1/100 [00:26<44:25, 26.92s/it]

epoch  0 loss 207.64 training accuracy 0.49


  2%|▏         | 2/100 [00:43<34:01, 20.83s/it]

epoch  1 loss 159.16 training accuracy 0.62


  3%|▎         | 3/100 [00:59<30:21, 18.78s/it]

...

epoch  97 loss 23.37 training accuracy 0.95


 99%|█████████▉| 99/100 [26:39<00:16, 16.41s/it]

epoch  98 loss 23.23 training accuracy 0.95


100%|██████████| 100/100 [26:55<00:00, 16.16s/it]

epoch  99 loss 23.09 training accuracy 0.95


The loss keeps *decreasing*, so that is encouraging. We need to run more epochs! We can also play with the learning rates!

Verify with the test set:

In [ ]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [53]:
y_test_one_hot = np.zeros((y_test.shape[0], 10), dtype=np.float32)

for i, val in enumerate(y_test):
    y_test_one_hot[i, val] = 1.0
    
print(y_test_one_hot)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
X_test.shape, y_test.shape, y_test_one_hot.shape

((10000, 784), (10000,), (10000, 10))

To predict the test dataset, we run one feedforward pass with a batch of size 10,000!

In [ ]:
a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(X_test)
loss = compute_loss(a4, y_test_one_hot)
predictions = predict(a4)
predictions

<ipython-input-27-a5da05856836>:6: RuntimeWarning: overflow encountered in exp
  a2 = 1/(1 + np.exp(-z2))


array([7, 2, 1, ..., 4, 5, 6])

In [ ]:
labels = predict(y_test_one_hot)
labels

array([7, 2, 1, ..., 4, 5, 6])

In [ ]:
wrong = np.where(predictions != labels, np.matrix([1.]), np.matrix([0.]))
test_accuracy = 1 - (np.sum(wrong) / 10000)
test_accuracy

0.9453

## Set the hyperparameter learning rate = 0.001, epoch = 100:

In [14]:
eta=0.001
alpha=0.001
batch_size=50
n_iter = 100

### Reinitialize the parameter

Initialize the number of neurons per layer:

In [41]:
n_hidden_1, n_hidden_2, n_output = 100, 100, 10
n_input = len(X_train[0,0,:]) #returns the flattened image size (28*28 = 784)
n_input

784

Initialize the weight1, weight2, weight3

In [42]:
w1, w2, w3 = init_weights(n_input, n_hidden_1, n_hidden_2, n_output, batch_size)

Initialize Markovian weight history, losses and accuracy per epoch:

In [43]:
delta_w1_prev = np.zeros(w1.shape)
delta_w2_prev = np.zeros(w2.shape)
delta_w3_prev = np.zeros(w3.shape)

train_losses = []
train_acc = []

In [ ]:
from tqdm import tqdm
#epoch loop
for i in tqdm(range(n_iter)):
    # batch loop
    for (input, label) in zip(X_train, y_train):
        one_hot_label = one_hot_enc_v2(label, num_labels=10)

        a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(input)
        loss = compute_loss(a4, one_hot_label)
        grad1, grad2, grad3 = compute_backward_pass_fullmath([a1, z2, a2, z3, a3, z4, a4], one_hot_label)

        # multiplicative learning factor
        delta_w1, delta_w2, delta_w3 = eta * grad1, eta * grad2, eta * grad3

        # additive learning factor
        w1 = w1 + delta_w1 + delta_w1_prev * alpha
        w2 = w2 + delta_w2 + delta_w2_prev * alpha
        w3 = w3 + delta_w3 + delta_w3_prev * alpha

        delta_w1_prev, delta_w2_prev, delta_w3_prev = delta_w1, delta_w2, delta_w3

        train_losses.append(loss)
        predictions = predict(a4)

        wrong = np.where(predictions != label, np.matrix([1.]), np.matrix([0.]))
        accuracy = 1 - (np.sum(wrong) / batch_size)
        train_acc.append(accuracy)

    # epoch loss and accuracy (mean of al batches)
    print('epoch ', i, 'loss %.2f' % np.mean(np.matrix(train_losses)).item(), 
          'training accuracy %.2f' % np.mean(np.matrix(train_acc)).item())

In [ ]:
a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(X_test)
loss = compute_loss(a4, y_test_one_hot)
predictions = predict(a4)
predictions

<ipython-input-27-a5da05856836>:6: RuntimeWarning: overflow encountered in exp
  a2 = 1/(1 + np.exp(-z2))


array([7, 2, 1, ..., 9, 8, 6])

In [ ]:
labels = predict(y_test_one_hot)
labels

array([7, 2, 1, ..., 4, 5, 6])

In [ ]:
wrong = np.where(predictions != labels, np.matrix([1.]), np.matrix([0.]))
test_accuracy = 1 - (np.sum(wrong) / 10000)
test_accuracy

0.7429

## Set the hyperparameter learning rate = 0.01, epoch = 50:

In [45]:
eta=0.01
alpha=0.01
batch_size=50
n_iter = 50

### Reinitialize the parameter

Initialize the number of neurons per layer:

In [47]:
n_hidden_1, n_hidden_2, n_output = 100, 100, 10
n_input = len(X_train[0,0,:]) #returns the flattened image size (28*28 = 784)
n_input

784

Initialize the weight1, weight2, weight3

In [48]:
w1, w2, w3 = init_weights(n_input, n_hidden_1, n_hidden_2, n_output, batch_size)

Initialize Markovian weight history, losses and accuracy per epoch:

In [49]:
delta_w1_prev = np.zeros(w1.shape)
delta_w2_prev = np.zeros(w2.shape)
delta_w3_prev = np.zeros(w3.shape)

train_losses = []
train_acc = []

In [50]:
from tqdm import tqdm
#epoch loop
for i in tqdm(range(n_iter)):
    # batch loop
    for (input, label) in zip(X_train, y_train):
        one_hot_label = one_hot_enc_v2(label, num_labels=10)

        a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(input)
        loss = compute_loss(a4, one_hot_label)
        grad1, grad2, grad3 = compute_backward_pass_fullmath([a1, z2, a2, z3, a3, z4, a4], one_hot_label)

        # multiplicative learning factor
        delta_w1, delta_w2, delta_w3 = eta * grad1, eta * grad2, eta * grad3

        # additive learning factor
        w1 = w1 + delta_w1 + delta_w1_prev * alpha
        w2 = w2 + delta_w2 + delta_w2_prev * alpha
        w3 = w3 + delta_w3 + delta_w3_prev * alpha

        delta_w1_prev, delta_w2_prev, delta_w3_prev = delta_w1, delta_w2, delta_w3

        train_losses.append(loss)
        predictions = predict(a4)

        wrong = np.where(predictions != label, np.matrix([1.]), np.matrix([0.]))
        accuracy = 1 - (np.sum(wrong) / batch_size)
        train_acc.append(accuracy)

    # epoch loss and accuracy (mean of al batches)
    print('epoch ', i, 'loss %.2f' % np.mean(np.matrix(train_losses)).item(), 
          'training accuracy %.2f' % np.mean(np.matrix(train_acc)).item())

  2%|▏         | 1/50 [00:16<13:48, 16.92s/it]

epoch  0 loss 322.52 training accuracy 0.43


  4%|▍         | 2/50 [00:33<13:11, 16.49s/it]

epoch  1 loss 224.86 training accuracy 0.57


  6%|▌         | 3/50 [00:49<12:54, 16.49s/it]

...

epoch  47 loss 94.36 training accuracy 0.84


 98%|█████████▊| 49/50 [13:44<00:16, 16.64s/it]

epoch  48 loss 94.18 training accuracy 0.85


100%|██████████| 50/50 [14:01<00:00, 16.82s/it]

epoch  49 loss 94.01 training accuracy 0.85


In [54]:
a1, z2, a2, z3, a3, z4, a4 = compute_forward_pass(X_test)
loss = compute_loss(a4, y_test_one_hot)
predictions = predict(a4)
predictions

<ipython-input-31-a5da05856836>:6: RuntimeWarning: overflow encountered in exp
  a2 = 1/(1 + np.exp(-z2))


array([3, 2, 1, ..., 4, 5, 6])

In [55]:
labels = predict(y_test_one_hot)
labels

array([7, 2, 1, ..., 4, 5, 6])

In [56]:
wrong = np.where(predictions != labels, np.matrix([1.]), np.matrix([0.]))
test_accuracy = 1 - (np.sum(wrong) / 10000)
test_accuracy

0.8439

## Conclusion

* For each result, we can see the hyperparameter with learning rate = 0.01, batch_size=50, epoch = 100 has the highest accuracy, the accuracy = 95% and the test accuracy = 100%